In [1]:
import pandas as pd

import codecs

import folium
import folium.plugins as plugins

from math import log

import warnings
warnings.filterwarnings("ignore")

In [2]:
def popup_html(data):
    html = '<font face="Helvetica" weight=400><h3>{}</h3>'.format(data.selsoviet)
    html += '<p><b>{}</b><br>'.format(data.district)
    html += '<b>{}:</b> {} чел</p>'.format(name, data.value)
    return html

In [3]:
filename = 'data_selsoviet.xlsx'
df = pd.read_excel(filename)
df

,district,selsoviet,value
0,Воловский район,Большеивановский сельсовет,436
1,Воловский район,Большовский сельсовет,1333
2,Воловский район,Васильевский сельсовет,538
3,Воловский район,Верхнечесноченский сельсовет,549
4,Воловский район,Воловский сельсовет,3251
...,...,...,...
289,Грязинский район,Грязи,46683
290,Задонский район,Задонск,9527
291,Лебедянский район,Лебедянь,19203
292,Усманский район,Усмань,19625


In [4]:
lg = []
max_log = log(df.value.max())

for index in df.index:
    lg.append(max_log - log(df.loc[index].value))
    
df['log'] = lg

In [5]:
m = folium.Map([52.7, 39.1], zoom_start=8, tiles='cartodbpositron', control_scale=True)

filename = r'J:\~ 09_Машинное обучение_Прогноз показателей СЭР\Геограницы\Обновленные\Сельсоветы ЛО.geojson'
name = "Численность населения"

folium.Choropleth(
    geo_data=open(filename, encoding='utf-8').read(),
    name=name,
    data=df,
    key_on='feature.properties.name',
    columns=['selsoviet', 'log'],
    fill_color='RdYlGn',
    line_weight=1,
    fill_opacity=0.5,
    line_opacity=0.2,
).add_to(m)

for index in df.index:
    filename = r'J:\~ 09_Машинное обучение_Прогноз показателей СЭР\Геограницы\Обновленные\Сельсоветы ЛО\{}\{}.geojson'.format(df.loc[index].district, df.loc[index].selsoviet)
    geoj = folium.GeoJson(data=open(filename, encoding='utf-8').read(), control=False,
                          style_function = lambda x: {'color': 'black', 'fillOpacity' : 0, 'opacity' : 0.1, 'weight': 1},
                          highlight_function=lambda x: {'color': 'black', 'fillOpacity' : 0.25, 'opacity' : 0.1, 'weight': 1},
                          smooth_factor=0)
    iframe = folium.IFrame(popup_html(df.loc[index]), width=250, height=150)
    popup = folium.Popup(iframe)
    popup.add_to(geoj)
    geoj.add_to(m)

plugins.Fullscreen(position='topleft', title='На весь экран', title_cancel='Уменьшить', force_separate_button=True).add_to(m)

m.save('{} карта по сельсоветам ЛО.html'.format(name))